# Current Problem: Shift Variance
Currently, it seems that shifting an input image causes different deconvolution behaviour; the neural network is shift-variant. I've also seen that the PSFs get all messed up and disjointed. I want to see whether this shifting happens at some point during the creation of the neural network.

Here are the things to check in initialization:
1. Are the padded PSFs, passed into the builder function, centered?
2. ~~Are the PSFs shifted insided the builder function?~~ I already know they aren't from prior investigation. `hdiff`==`wdiff`==0.

In [1]:
import preset_nn
from matplotlib import pyplot as plt

from pdb import set_trace
set_trace()

model = preset_nn.instant_multiwienernet()

2023-02-08 09:48:41.568264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 09:48:41.933740: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-08 09:48:43.906556: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dshteinbok/miniconda3/lib/:/home/dshteinbok/miniconda3/lib/:/home/dshteinbok/miniconda3/envs/tf/lib/
2023-02-08 09:48:43.907639: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not lo

--Return--
None
> /tmp/ipykernel_181682/866717088.py(5)<cell line: 5>()
      3 
      4 from pdb import set_trace
----> 5 set_trace()
      6 
      7 model = preset_nn.instant_multiwienernet()

ipdb> c


2023-02-08 09:48:50.649296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 09:48:53.913735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18275 MB memory:  -> device: 0, name: NVIDIA RTX A4500, pci bus id: 0000:01:00.0, compute capability: 8.6
2023-02-08 09:48:53.918953: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 5 MB memory:  -> device: 1, name: NVIDIA RTX A4500, pci bus id: 0000:2c:00.0, compute capability: 8.6
2023-02-08 09:48:53.920068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 

Above, I verified that an (800, 1280)-shaped initial PSF does not get padded at all. This is the situation we had in `training_code.ipynb`.

I also verified, in `training_code.ipynb` that the PSFs really are centered after padding (before they are passed into the builder function). This eliminates the suspicion of both 1. and 2. above.

The next "section" that could be suspect is the actual behavior when calling the network on some input. Namely, the following bits of code are suspect:
1. `layers.py:32`, which pads the input image
2. `layers.py:48`, which pads the PSF
3. `layers.py:61`, which crops the now-padded output image.

In [7]:
import imageio
import sys
sys.path.append("/home/dshteinbok/denoising_experiments/")
import denoising
test_img = imageio.imread('../data/nV3_mastermat_probe_unshifted/100.png')

['/home/dshteinbok/MultiWienerNet/tensorflow', '/home/dshteinbok/miniconda3/envs/tf/lib/python39.zip', '/home/dshteinbok/miniconda3/envs/tf/lib/python3.9', '/home/dshteinbok/miniconda3/envs/tf/lib/python3.9/lib-dynload', '', '/home/dshteinbok/miniconda3/envs/tf/lib/python3.9/site-packages', '/home/dshteinbok/denoising_experiments/']


ImportError: ERROR: recursion is detected during loading of "cv2" binary extensions. Check OpenCV installation.

In [4]:
model.load_weights("saved_models/multiwiener_nV3_probe_noisy2/model_weights")

In [8]:
set_trace()
model(denoising.rescale_to_one(test_img).reshape((1, 800, 1280, 1)))

--Return--
None
> /tmp/ipykernel_181682/2284416396.py(1)<cell line: 1>()
----> 1 set_trace()
      2 model(denoising.rescale_to_one(test_img).reshape((1, 800, 1280, 1)))

ipdb> b 2
Breakpoint 2 at /tmp/ipykernel_181682/2284416396.py:2
ipdb> c
    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

None
> /tmp/ipykernel_181682/2284416396.py(2)<cell line: 2>()
      1 set_trace()
2---> 2 model(denoising.rescale_to_one(test_img).reshape((1, 800, 1280, 1)))

ipdb> s
NameError: name 'denoising' is not defined
None
> /tmp/ipykernel_181682/2284416396.py(2)<cell line: 2>()
      1 set_trace()
2---> 2 model(denoising.rescale_to_one(test_img).reshape((1, 800, 1280, 1)))

ipdb> q


due to weird errors above, I just went ahead and did the same inside `Complete image processing.ipynb`. From that, I draw the following conclusions:
1. padding at layers.py:32 is  `((0, 0), (400, 400), (640, 640), (0, 0))`
2. `padding_psf` at layers.py:48 is `((400, 400), (640, 640), (0, 0))`
3. h = 1600, w = 2560, h1 = 400, h2 = 1200, w1 = 640, w2 = 1920 in `crop_2d_tf` as called from `layers.py:61`. This is in accordance with what I expected.
Nonetheless, we see ringing at the edge of the image. Not quite sure why...

I noticed the ringing at the edge of the image is absent in the test images from Laura Waller. However, it is present in the case of the bandpass-filtered cell images. I suspect this is because the images I was looking at from Laura Waller have a zero-valued background, so the zero padding does not cause a discontinuity at the edge of the padded image. On the contrary, the bandpassed neuron images have a nonzero background throughout, and so the zero-padding would cause a jump "discontinuity". When Fourier-transforming and then inverse-Fourier-transforming, we would end up with Gibbs ringing at the edges. When multiplying with a spatial frequency in the Fourier domain (causing a shift in the image domain, e.g. to the top left), this ringing is shifted up to the top-left as well.

The theory that this is caused by the non-zero background is supported by slide 211: the 70th frame of the cell video. Here, you can see that the ringing is absent at the edge where the background is dark. You can also see the bottom of the "ringing" is darker; this is the other side of the Gibbs phenomenon (on the lower side of the jump) which has been shifted up.

The problem is that this doesn't explain the shift variance of the neural network acting on the image. That is the bigger problem!

Playing around with the neural network, I actually came up with a hypothesis for the shift-variance:
* it's a consequence of the U-net, not the MultiWienerNet
    * This is supported by the fact that, having zoomed in on part of the image that wouldn't be affected by the differences at the edge (i.e. proximity of the ROI to zero-padding), the image post-multiwiener-layer seems almost the same regardless of whether it was shifted or not before application of the NN.
* it's a consequence of proximity to the edges (the center of a shifted and unshifted image through the whole NN looks almost the same)
* it happens in the later layers of the U-net (since the top left part, which would be placed right next to zero-padding in the unshifted image, looks different from the same feature in the shifted image, which is again next to a bunch of zeros).